# Ensemble - Regression

## 보스톤 집값 예측

### 패키지 로딩

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd

### 데이터 로드 및 확인

In [2]:
boston = pd.read_csv('./dataset/HousingData.csv')

In [4]:
# print(boston.isna().sum())
# boston.ffill(inplace=True)  # 발견된 결측치의 근처 값(앞에 값)으로 채우기

### 데이터 분할

In [5]:
x = boston.drop('MEDV', axis=1)
y = boston['MEDV']

In [6]:
scaled_x = StandardScaler().fit_transform(x)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(scaled_x, y, test_size=0.3, random_state=10)

### 모델 생성 및 검증

In [8]:
# n_estimators: 트리의 개수
# random_state: 데이터 샘플링 시 사용되는 시드 값
model = RandomForestRegressor(n_estimators=200, random_state=0)
model.fit(x_train, y_train)

RandomForestRegressor(n_estimators=200, random_state=0)

In [9]:
y_hat = model.predict(x_test)
print(f'MAE:{mean_absolute_error(y_test, y_hat):.3f}')
print(f'RMSE:{np.sqrt(mean_squared_error(y_test, y_hat)):.3f}')
print(f'R2 Score:{r2_score(y_test, y_hat):.3f}')

MAE:2.534
RMSE:3.533
R2 Score:0.872


In [10]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(x_train, y_train)

LinearRegression()

In [11]:
y_hat = lm.predict(x_test)
print(f'MAE:{mean_absolute_error(y_test, y_hat):.3f}')
print(f'RMSE:{np.sqrt(mean_squared_error(y_test, y_hat)):.3f}')
print(f'R2 Score:{r2_score(y_test, y_hat):.3f}')

MAE:3.684
RMSE:5.432
R2 Score:0.698


### 파라메터 튜닝

In [14]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[10, 20, 50, 100, 200, 300],
    'bootstrap':[True, False],
    'max_features':[2, 4, 6, 8, 13]
}
model = RandomForestRegressor()
gs = GridSearchCV(model, params, cv=5, scoring='neg_mean_squared_error')
gs.fit(scaled_x, y)
print('최적 파라메터 조합:', gs.best_params_)

최적 파라메터 조합: {'bootstrap': True, 'max_features': 6, 'n_estimators': 50}


In [15]:
y_hat = gs.best_estimator_.predict(x_test)
print(f'MAE:{mean_absolute_error(y_test, y_hat):.3f}')
print(f'RMSE:{np.sqrt(mean_squared_error(y_test, y_hat)):.3f}')
print(f'R2 Score:{r2_score(y_test, y_hat):.3f}')

MAE:0.927
RMSE:1.304
R2 Score:0.983


# Bagging - Classifier

## 와인 품종 분류

### 패키지 로딩

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, classification_report

import numpy as np
import pandas as pd

### 데이터 로딩 및 확인 

In [2]:
wine = load_wine()
# print(wine.DESCR)
x = wine.data
y = wine.target
df = pd.DataFrame(x, columns=wine.feature_names)
df['class'] = y
display(df.head())

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,class
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [3]:
print(df['class'].value_counts())

class
1    71
0    59
2    48
Name: count, dtype: int64


In [4]:
print(df.describe())

          alcohol  malic_acid         ash  alcalinity_of_ash   magnesium  \
count  178.000000  178.000000  178.000000         178.000000  178.000000   
mean    13.000618    2.336348    2.366517          19.494944   99.741573   
std      0.811827    1.117146    0.274344           3.339564   14.282484   
min     11.030000    0.740000    1.360000          10.600000   70.000000   
25%     12.362500    1.602500    2.210000          17.200000   88.000000   
50%     13.050000    1.865000    2.360000          19.500000   98.000000   
75%     13.677500    3.082500    2.557500          21.500000  107.000000   
max     14.830000    5.800000    3.230000          30.000000  162.000000   

       total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
count     178.000000  178.000000            178.000000       178.000000   
mean        2.295112    2.029270              0.361854         1.590899   
std         0.625851    0.998859              0.124453         0.572359   
min         0.9

### 데이터 분할

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaled_x = MinMaxScaler().fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(scaled_x, y, test_size=0.3, random_state=0, stratify=y)

### 모델 생성
- RandomForestClassifier 파라메터
    - bootstrap: 부트스트랩 사용 여부(default=True), True이면 복원 추출된 샘플로 학습, False면 표본 전체 이용해 학습
    - criterion: 트리 분할에 사용할 알고리즘 (entropy, gini(불평등지수))
    - max_depth: 나무의 최대 깊이 (과적합 제어 용도)
    - min_samples_split: 노드를 분할하는 데 필요한 최소 샘플 수 (샘플 수 커질수록 깊이 깊어짐)
    - n_estimators: 사용할 트리의 개수 (default=100)

In [6]:
model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier()

### 모델 평가

In [7]:
y_hat = model.predict(x_test)
print(f'정확도:{accuracy_score(y_test, y_hat):.3f}')
report = classification_report(y_test, y_hat)
print(report)

정확도:1.000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00        15

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54



In [23]:
model = RandomForestClassifier(n_estimators=400, min_samples_split=3, bootstrap=False)
model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=False, min_samples_split=3, n_estimators=400)

### 트리 시각화

In [24]:
from sklearn.tree import export_graphviz

estimator = model.estimators_[0]
export_graphviz(estimator, out_file='tree.dot',
                feature_names=wine.feature_names,
                class_names=wine.target_names,
                precision=3, # 소수점 정밀도
                filled=True, # Class별 노드의 색깔 채우기
                 rounded=True # 노드의 박스 모양을 둥글게
               )

- graphviz 패키지 설치

In [10]:
# !pip install graphviz

In [25]:
import graphviz

with open('tree.dot') as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)

# bootstrap 방식 사용해 복원 추출 했기 때문에 샘플의 개수 상이
# sample 수 3개 남을 때까지 분할 -> sample=3 인 것도 마지막까지 분할해 줌

In [18]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

plt.figure(figsize=(30, 20))
plot_tree(estimator,
         feature_names=wine.feature_names,
         class_names=wine.target_names,
         filled=True,
         fontsize=13)
plt.show()

### 하이퍼 파라메터 튜닝

In [35]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth':[3,5,8,12,15,20],
    'min_samples_split':[2,3,6,20,25],
    'criterion':['gini','entropy']
}
model = RandomForestClassifier()
gs = GridSearchCV(model, params, cv=5, scoring='accuracy')
gs.fit(scaled_x, y)
print('최적의 파라메터:', gs.best_params_)
print('최적의 파라메터 정확도:', gs.best_score_)

최적의 파라메터: {'criterion': 'entropy', 'max_depth': 12, 'min_samples_split': 3}
최적의 파라메터 정확도: 0.9888888888888889


In [36]:
best_estimator = gs.best_estimator_
y_hat = best_estimator.predict(x_test)
print(f'정확도:{accuracy_score(y_test, y_hat):.3f}')

정확도:1.000


### Feature Importance
- 의사결정나무 알고리즘이 학습을 통해 규칙을 정하는 데 있어 피처의 중요도를 나타내는 지표
- 일반적으로 같이 높을수록 해당 피처의 중요도가 높다는 의미로 해석

In [37]:
print(estimator.feature_importances_)

[0.06994738 0.05550687 0.01728016 0.01840491 0.02617587 0.23457195
 0.         0.         0.         0.01338539 0.16656346 0.0455877
 0.35257631]


In [38]:
importance = { k:v for k, v in zip(wine.feature_names, estimator.feature_importances_) }
df_importance = pd.DataFrame(pd.Series(importance), columns=['Importance'])
df_importance = df_importance.sort_values('Importance', ascending=False)
display(df_importance)

,Importance
proline,0.352576
total_phenols,0.234572
hue,0.166563
alcohol,0.069947
malic_acid,0.055507
od280/od315_of_diluted_wines,0.045588
magnesium,0.026176
alcalinity_of_ash,0.018405
ash,0.017280
color_intensity,0.013385


In [39]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.barplot(data=df_importance, x = 'Importance', y = df_importance.index)
plt.show()